In [92]:
import json, csv, re, argparse, sys, math
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
import pickle

from PIL import Image, ImageDraw, ImageFont
import textwrap
import pickle
import unicodedata
import requests
from io import BytesIO  # [FIX] load_image_any 需要 BytesIO


In [95]:
# -------------------- 小工具 --------------------
def safe_str(x): 
    """NaN 轉空字串，再轉成 str"""
    return "" if (x is None or (isinstance(x, float) and pd.isna(x))) else str(x)

def count_digits(s): 
    """計算字串中的數字個數"""
    return sum(ch.isdigit() for ch in safe_str(s))

def read_json_file(path: Path):
    """讀 JSON（加上存在與空檔檢查）"""
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    text = path.read_text(encoding="utf-8").strip()
    if not text:
        raise ValueError(f"File is empty: {path}")
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        preview = text[:200]
        raise ValueError(f"Invalid JSON in {path} (preview: {preview!r})") from e

def parse_ls_rect(item):
    """
    從 Label Studio 的 result item 取出：
      - label: 類別名
      - rect_pct: {x,y,w,h} 百分比(0~100)
      - ow, oh: 原圖寬高（int）
    """
    v = item["value"]
    label = v["rectanglelabels"][0] if isinstance(v.get("rectanglelabels"), list) else v.get("labels", [""])[0]
    rect_pct = dict(x=v["x"], y=v["y"], w=v["width"], h=v["height"])
    ow = item.get("original_width") or item.get("image_original_width")
    oh = item.get("original_height") or item.get("image_original_height")
    if ow is None or oh is None:
        raise KeyError("Label Studio item missing original_width/original_height.")
    return label, rect_pct, int(ow), int(oh)

def pct_to_px(rpct, ow, oh):
    """百分比座標 → 像素座標（以原圖 ow,oh 為基）"""
    return {
        "x": rpct["x"] / 100.0 * ow,
        "y": rpct["y"] / 100.0 * oh,
        "w": rpct["w"] / 100.0 * ow,
        "h": rpct["h"] / 100.0 * oh,
    }

def resize_rect(rect, from_w, from_h, to_w, to_h):
    """把矩形框從原圖尺寸等比映射到 template 尺寸"""
    sx, sy = to_w / float(from_w), to_h / float(from_h)
    return {
        "x": rect["x"] * sx,
        "y": rect["y"] * sy,
        "w": rect["w"] * sx,
        "h": rect["h"] * sy,
    }

def delta_from(gt, init, W, H):
    """
    計算 offset：
      dx = (x_gt - x0) / W
      dy = (y_gt - y0) / H
      dlogw = log(w_gt / w0)
      dlogh = log(h_gt / h0)
    """
    w0 = max(1e-6, float(init["w"]))
    h0 = max(1e-6, float(init["h"]))
    return {
        "dx":   (float(gt["x"]) - float(init["x"])) / float(W),
        "dy":   (float(gt["y"]) - float(init["y"])) / float(H),
        "dlogw": float(np.log(max(1e-6, float(gt["w"])) / w0)),
        "dlogh": float(np.log(max(1e-6, float(gt["h"])) / h0)),
    }

def build_features(meta_row: dict):
    """
    從 meta.csv 的一列資料建特徵（可自行擴充）
    欄位建議：title, bullets, vip_price, unit_price, non_vip_price, has_badge
    """
    title  = safe_str(meta_row.get("title", ""))
    bullets = safe_str(meta_row.get("bullets", ""))
    vip    = safe_str(meta_row.get("vip_price", ""))
    unitp  = safe_str(meta_row.get("unit_price", ""))
    nonvip = safe_str(meta_row.get("non_vip_price", ""))
    has_badge = int(str(meta_row.get("has_badge", 0)).strip() not in ["", "0", "False", "false"])

    feats = {
        "title_len": len(title),
        "bullets_len": len(bullets),
        "bullets_lines": len([b for b in re.split(r"[;\n]", bullets) if b.strip()]),
        "vip_digits": count_digits(vip),
        "unit_digits": count_digits(unitp),
        "nonvip_digits": count_digits(nonvip),
        "has_badge": has_badge,
    }
    return feats

# -------------------- 主流程：產生訓練 CSV --------------------
def run_preprocess(label_studio_json: Path, initial_boxes_json: Path, out_csv: Path, meta_csv: Path|None=None):
    # 讀 initial boxes（baseline）
    init_cfg = read_json_file(initial_boxes_json)
    Tw = int(init_cfg["canvas"]["width"])
    Th = int(init_cfg["canvas"]["height"])
    init_boxes = init_cfg["boxes"]             # dict: class -> {x,y,w,h}
    classes = list(init_boxes.keys())          # 固定輸出順序以 initial_boxes 為準

    # 讀 meta（可選）
    meta_df = None
    if meta_csv and Path(meta_csv).exists():
        meta_df = pd.read_csv(meta_csv)

    # 讀 Label Studio 匯出（Common JSON）
    tasks = read_json_file(label_studio_json)

    rows = []
    for t in tasks:
        img_field = t.get("file_upload") or t.get("data", {}).get("image") or t.get("id")
        image_id = Path(str(img_field)).stem

        annos = t.get("annotations") or t.get("completions") or []
        if not annos:
            continue
        res = annos[0].get("result", [])

        # 收集每一類的 GT（縮放到 template 尺寸）
        gt_scaled = {}
        for r in res:
            if r.get("type") not in ("rectanglelabels", "rectangles"):
                continue
            label, rpct, ow, oh = parse_ls_rect(r)
            if label not in classes:
                continue
            rect_px = pct_to_px(rpct, ow, oh)
            rect_tpl = resize_rect(rect_px, from_w=ow, from_h=oh, to_w=Tw, to_h=Th)
            gt_scaled[label] = rect_tpl

        # 準備一行輸出
        row = {"image_id": image_id}

        # 合併 meta 特徵
        if meta_df is not None and "image_id" in meta_df.columns:
            m = meta_df.loc[meta_df["image_id"] == image_id]
            feats = build_features(m.iloc[0].to_dict()) if not m.empty else build_features({})
        else:
            feats = build_features({})
        row.update(feats)

        # 依 classes 順序寫 offsets
        for cls in classes:
            init = init_boxes[cls]
            gt = gt_scaled.get(cls)
            if gt is None:
                d = {"dx": 0.0, "dy": 0.0, "dlogw": 0.0, "dlogh": 0.0}
            else:
                d = delta_from(gt, init, Tw, Th)
            row[f"{cls}_dx"]    = d["dx"]
            row[f"{cls}_dy"]    = d["dy"]
            row[f"{cls}_dlogw"] = d["dlogw"]
            row[f"{cls}_dlogh"] = d["dlogh"]

        rows.append(row)

    # 輸出 CSV
    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False, quoting=csv.QUOTE_MINIMAL)
    print(f"[OK] Saved {out_csv}  rows={len(df)}")
    print("Columns:", list(df.columns))

    # [NEW] 同步存下「訓練時」的類別順序，供推論讀回
    class_order_path = Path(out_csv).with_suffix(".classes.json")  # e.g., offsets_for_tf.classes.json
    with open(class_order_path, "w", encoding="utf-8") as f:
        json.dump(classes, f, ensure_ascii=False, indent=2)
    print(f"[INFO] Saved class order -> {class_order_path}")
    return df

# -------------------- 入口（命令列 / Notebook 皆可） --------------------
def main_cli():
    """命令列用法：python preprocess_layout_v3.py --label_studio_json ... --initial_boxes_json ... --out_csv ... [--meta_csv ...]"""
    ap = argparse.ArgumentParser()
    ap.add_argument("--label_studio_json", required=True, help="Label Studio Common JSON export")
    ap.add_argument("--initial_boxes_json", required=True, help="Initial boxes JSON (template baseline)")
    ap.add_argument("--out_csv", required=True, help="Output CSV path")
    ap.add_argument("--meta_csv", default="", help="(Optional) meta CSV with image_id + product fields")
    args = ap.parse_args()

    run_preprocess(
        label_studio_json=Path(args.label_studio_json),
        initial_boxes_json=Path(args.initial_boxes_json),
        out_csv=Path(args.out_csv),
        meta_csv=Path(args.meta_csv) if args.meta_csv else None
    )

# 若在命令列執行：走 argparse
if __name__ == "__main__" and not hasattr(sys.modules["__main__"], "__file__"):
    pass
elif __name__ == "__main__":
    main_cli()

# -------------------- 下面開始：訓練 + 產圖 --------------------
from pathlib import Path
df_FeatureEngineering = run_preprocess(
    label_studio_json=Path("Objects_positions.json"),
    initial_boxes_json=Path("Initial_boxes.json"),
    out_csv=Path("offsets_for_tf.csv"),
    meta_csv=None
)
df_FeatureEngineering.head(15)

# =====Model Building=====
df = pd.read_csv("offsets_for_tf.csv")

# features (X) = 除了 image_id 以外的欄位，且不含 offsets
X = df.drop(columns=[c for c in df.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh")) or c=="image_id"])
# targets (Y) = offsets
Y = df[[c for c in df.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh"))]]

# 切分資料
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# 標準化（重要，避免不同尺度影響）
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(Y_train.shape[1])  # 輸出數量 = offsets 欄位數
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='mse',   # 回歸問題
    metrics=[MeanSquaredError(), MeanAbsoluteError()]
)

# 訓練
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=50,
    batch_size=16
)

# 假設有一筆新產品的 meta feature
sample = X_val[0:1]
pred_offset = model.predict(sample)
print("Predicted offsets (sample):", pred_offset[:1])

# 存模型 & scaler
model.save("layout_model.keras")   # 存成 Keras 格式
print("Model saved to layout_model.keras")
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


[OK] Saved offsets_for_tf.csv  rows=29
Columns: ['image_id', 'title_len', 'bullets_len', 'bullets_lines', 'vip_digits', 'unit_digits', 'nonvip_digits', 'has_badge', 'Sales Period Position_dx', 'Sales Period Position_dy', 'Sales Period Position_dlogw', 'Sales Period Position_dlogh', 'Brand Position_dx', 'Brand Position_dy', 'Brand Position_dlogw', 'Brand Position_dlogh', 'Product Position_dx', 'Product Position_dy', 'Product Position_dlogw', 'Product Position_dlogh', 'FAB Position_dx', 'FAB Position_dy', 'FAB Position_dlogw', 'FAB Position_dlogh', 'Product Image Position_dx', 'Product Image Position_dy', 'Product Image Position_dlogw', 'Product Image Position_dlogh', 'VIP Price Position_dx', 'VIP Price Position_dy', 'VIP Price Position_dlogw', 'VIP Price Position_dlogh', 'Non VIP Price Position_dx', 'Non VIP Price Position_dy', 'Non VIP Price Position_dlogw', 'Non VIP Price Position_dlogh', 'Original Price Position_dx', 'Original Price Position_dy', 'Original Price Position_dlogw', 'Ori

/opt/anaconda3/envs/boardclick/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - loss: 0.0680 - mean_absolute_error: 0.1592 - mean_squared_error: 0.0680 - val_loss: 0.0647 - val_mean_absolute_error: 0.1542 - val_mean_squared_error: 0.0647
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0675 - mean_absolute_error: 0.1581 - mean_squared_error: 0.0675 - val_loss: 0.0643 - val_mean_absolute_error: 0.1535 - val_mean_squared_error: 0.0643
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0670 - mean_absolute_error: 0.1571 - mean_squared_error: 0.0670 - val_loss: 0.0639 - val_mean_absolute_error: 0.1528 - val_mean_squared_error: 0.0639
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0665 - mean_absolute_error: 0.1561 - mean_squared_error: 0.0665 - val_loss: 0.0634 - val_mean_absolute_error: 0.1523 - val_mean_squared_error: 0.0634
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0661 - mean_absolute_error: 0.1552 - mean_squared_error: 0.0661 - val_loss: 0.0630 - val_mean_absolute_error: 0.151

In [106]:
# ============ 基本路徑 ============
EXCEL_PATH = "Board Click SKU.xlsx"
SHEET_USER = "User_Input"
SHEET_IMG  = "SKU_Image"
SHEET_ICON = "SKU_Icon"

TEMPLATE_PATH = "sasa_pink_1280.png"
INITIAL_BOXES_PATH = "Initial_boxes.json"

MODEL_PATH = "layout_model.keras"
SCALER_PATH = "scaler.pkl"

OUTPUT_DIR = Path("out_cards_cn")
OUTPUT_DIR.mkdir(exist_ok=True)

# ============ 偵錯選項（可關閉） ============
DRAW_DEBUG_BOXES = True   # [NEW] 想看框是否飛出，可設 True
PRINT_FIRST_PRED = True   # [NEW] 印第一列的預測 delta
BLEND = 1              # [NEW] 降暴：只採用 35% 的模型偏移量

# ============ 欄位映射 ============
MAP_TEXT = {
    "Brand Position":          "Brand",
    "VIP Price Position":      "VIP價",
    "Non VIP Price Position":  "優惠價",
    "Original Price Position": "建議價",
    "Product Position":        "Product name",
    "Sales Period Position":   "優惠期",
    "FAB Position":            "FAB",
}
COL_PRODUCT_SKU = "Product SKU"
COL_ICON_NAME   = "Icon"

# 若你的欄位名不是 link，請改成正確的欄位名
SKU_IMG_SKU_COL   = "Product SKU"
SKU_IMG_LINK_COL  = "Image"
SKU_ICON_NAME_COL = "Icon"
SKU_ICON_LINK_COL = "Icon Image"

# ============ 字型（請把路徑改成你電腦實際有的檔） ============
FONT_REGULAR_PATH = Path.home() / "Library/Fonts/NotoSansCJKsc-Regular.otf"
FONT_BOLD_PATH    = Path.home() / "Library/Fonts/NotoSansCJKsc-Bold.otf"

def _load_font(path, size):
    return ImageFont.truetype(str(path), size)

# ============ 常用工具 ============
def _safe(s):
    return "" if (s is None or (isinstance(s, float) and pd.isna(s))) else str(s)

def _count_digits(s):
    return sum(ch.isdigit() for ch in _safe(s))

def _norm_key(x):
    s = "" if pd.isna(x) else str(x)
    s = unicodedata.normalize("NFKC", s)  # 全形 → 半形
    return s.strip().casefold()

def _drive_share_to_direct(u: str) -> str:
    # 轉 Google Drive 分享連結 → 直接下載
    if not u: return u
    m = re.search(r"/d/([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    m = re.search(r"[?&]id=([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    return u

def load_image_any(path_or_url: str):
    if not path_or_url:
        return None
    s = str(path_or_url).strip()
    try:
        if s.startswith("http://") or s.startswith("https://"):
            s2 = _drive_share_to_direct(s)
            resp = requests.get(s2, timeout=15)
            resp.raise_for_status()
            return Image.open(BytesIO(resp.content)).convert("RGBA")
        p = Path(s)
        if p.exists():
            return Image.open(p).convert("RGBA")
    except Exception as e:
        print("[WARN] cannot load image:", s, e)
    return None

# ============ 特徵工程（需與訓練時一致） ============
FEATURE_ORDER = [
    "title_len", "bullets_len", "bullets_lines",
    "vip_digits", "unit_digits", "nonvip_digits",
    "has_badge",
]

def build_features_from_row(row: pd.Series, resolved_icon_path: str) -> dict:
    title   = _safe(row.get(MAP_TEXT["Product Position"], ""))
    bullets = _safe(row.get(MAP_TEXT["FAB Position"], ""))
    vip     = _safe(row.get(MAP_TEXT["VIP Price Position"], ""))
    unitp   = _safe(row.get(MAP_TEXT["Non VIP Price Position"], ""))
    orig    = _safe(row.get(MAP_TEXT["Original Price Position"], ""))
    has_icon = int(Path(_safe(resolved_icon_path)).exists()) if resolved_icon_path else 0

    return {
        "title_len": len(title),
        "bullets_len": len(bullets),
        "bullets_lines": len([b for b in bullets.split(";") if b.strip()]),
        "vip_digits": _count_digits(vip),
        "unit_digits": _count_digits(unitp),
        "nonvip_digits": _count_digits(unitp),  # [FIX] 與訓練時一致：non_vip_price，而不是 orig(建議價)
        "has_badge": has_icon,
    }

# ============ 幾何 / 佈局 ============
def _clip(v, lo, hi):  # [NEW] 便於做邊界裁切
    return max(lo, min(hi, v))

def apply_offset(init_box, delta, W, H):
    dx, dy, dlogw, dlogh = map(float, delta)

    # [NEW] 位移限制（避免飛出半張圖；可依據你的資料再調整）
    dx = _clip(dx, -0.5, 0.5)
    dy = _clip(dy, -0.5, 0.5)

    # [NEW] 尺寸縮放限制（0.5x ~ 2x），用裁好的 dlog 區間再 exp
    sx = math.exp(_clip(dlogw, math.log(0.5), math.log(2.0)))
    sy = math.exp(_clip(dlogh, math.log(0.5), math.log(2.0)))

    x = init_box["x"] + dx * W
    y = init_box["y"] + dy * H
    w = max(24, init_box["w"] * sx)  # [NEW] 最小寬/高，避免太小導致無法排版
    h = max(24, init_box["h"] * sy)

    # [NEW] 把盒子裁進畫布，避免完全跑出可視區
    x = _clip(x, 0, W - w)
    y = _clip(y, 0, H - h)

    return {"x": x, "y": y, "w": w, "h": h}

def paste_image_into_box(canvas_rgba, path_or_url, box, padding=6):
    im = load_image_any(path_or_url)
    if im is None:
        print("[WARN] image not found:", path_or_url)
        return
    x, y, w, h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    w2, h2 = max(1, w - padding*2), max(1, h - padding*2)
    ratio = min(w2 / im.width, h2 / im.height)
    im = im.resize((max(1,int(im.width*ratio)), max(1,int(im.height*ratio))), Image.LANCZOS)
    ox = x + (w - im.width)//2
    oy = y + (h - im.height)//2
    canvas_rgba.alpha_composite(im, (ox, oy))

def draw_text_in_box(draw, text, box, font_path, max_font=64, min_font=16, align="left", line_spacing=1.15):
    if not text or str(text).strip()=="":
        return
    x, y, w, h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    text = str(text)

    for fs in range(max_font, min_font-1, -2):
        font = _load_font(font_path, fs)
        approx_chars = max(1, int(w / (fs * 0.55)))
        lines = []
        for raw in text.split("\n"):
            lines += (textwrap.wrap(raw, width=approx_chars) if approx_chars>1 else [raw])

        bboxes = [draw.textbbox((0,0), ln, font=font) for ln in lines]
        line_heights = [bb[3]-bb[1] for bb in bboxes]
        total_h = int(sum(line_heights) + (len(lines)-1)*fs*(line_spacing-1))
        if total_h <= h:
            cur_y = y + (h - total_h)//2
            for ln in lines:
                bb = draw.textbbox((0,0), ln, font=font)
                lw = bb[2]-bb[0]
                if align == "center":
                    cur_x = x + (w - lw)//2
                elif align == "right":
                    cur_x = x + (w - lw)
                else:
                    cur_x = x
                draw.text((cur_x, cur_y), ln, font=font, fill=(0,0,0,255))
                cur_y += int(fs * line_spacing)
            return
    # 放不下就畫第一行
    draw.text((x, y), text.split("\n")[0][:30]+"…", font=_load_font(font_path, min_font), fill=(0,0,0,255))

# [NEW] 偵錯：描框
def _stroke(draw, box, color=(0,0,0,255), width=2):
    x,y,w,h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    draw.rectangle([x,y,x+w,y+h], outline=color, width=width)

# ============ 載入模板 & 初始盒 & 模型 ============
def load_template_and_boxes():
    tpl = Image.open(TEMPLATE_PATH).convert("RGBA")
    tw, th = tpl.size
    init_cfg = json.loads(Path(INITIAL_BOXES_PATH).read_text(encoding="utf-8"))
    cw, ch = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    if (tw, th) != (cw, ch):
        sx, sy = tw/float(cw), th/float(ch)
        for _, r in init_cfg["boxes"].items():
            r["x"], r["y"] = r["x"]*sx, r["y"]*sy
            r["w"], r["h"] = r["w"]*sx, r["h"]*sy
        init_cfg["canvas"]["width"], init_cfg["canvas"]["height"] = tw, th
        print(f"[INFO] Scaled initial boxes to {tw}x{th}")
    return tpl, init_cfg

def load_model_and_scaler():
    model = None
    scaler = None
    if Path(MODEL_PATH).exists():
        model = tf.keras.models.load_model(MODEL_PATH)
    else:
        print("[WARN] MODEL not found → 使用 zero offsets")
    if Path(SCALER_PATH).exists():
        with open(SCALER_PATH, "rb") as f:
            scaler = pickle.load(f)
    else:
        print("[WARN] SCALER not found → 直接用原始特徵")
    return model, scaler

# ============ 主流程（推論 & 畫圖） ============
def main():
    # 讀 Excel
    xls = pd.ExcelFile(EXCEL_PATH)
    df_user = pd.read_excel(xls, sheet_name=SHEET_USER)
    df_img  = pd.read_excel(xls, sheet_name=SHEET_IMG)
    df_icon = pd.read_excel(xls, sheet_name=SHEET_ICON)

    # 建查表（SKU / ICON）
    img_lookup  = {_norm_key(r[SKU_IMG_SKU_COL]): str(r[SKU_IMG_LINK_COL]).strip()
                   for _, r in df_img.iterrows() if SKU_IMG_SKU_COL in r and SKU_IMG_LINK_COL in r}
    icon_lookup = {_norm_key(r[SKU_ICON_NAME_COL]): str(r[SKU_ICON_LINK_COL]).strip()
                   for _, r in df_icon.iterrows() if SKU_ICON_NAME_COL in r and SKU_ICON_LINK_COL in r}

    # 載入模板與模型
    template_rgba, init_cfg = load_template_and_boxes()
    W, H = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    INIT_BOXES = init_cfg["boxes"]

    # [NEW] 強制使用「訓練時」的類別順序，防止對錯框
    CLASS_ORDER_FILE = Path("offsets_for_tf.classes.json")
    if CLASS_ORDER_FILE.exists():
        train_classes = json.loads(CLASS_ORDER_FILE.read_text(encoding="utf-8"))
        if set(train_classes) != set(INIT_BOXES.keys()):
            raise ValueError(
                "Classes mismatch between training and inference.\n"
                f"Train: {train_classes}\nInfer: {list(INIT_BOXES.keys())}\n"
                "請確認 Initial_boxes.json 與訓練時一致（或重新產生資料與模型）。"
            )
        CLASSES = train_classes[:]  # [NEW]
    else:
        CLASSES = list(INIT_BOXES.keys())  # 後備

    model, scaler = load_model_and_scaler()

    printed_pred_once = False

    # 逐列生成
    for idx, row in df_user.iterrows():
        # 解析圖片路徑
        sku_key  = _norm_key(row.get(COL_PRODUCT_SKU, ""))
        icon_key = _norm_key(row.get(COL_ICON_NAME, ""))
        prod_img_path = img_lookup.get(sku_key, "")
        icon_img_path = icon_lookup.get(icon_key, "")

        # 特徵 → DataFrame（與訓練欄位同名同序）
        feats = build_features_from_row(row, resolved_icon_path=icon_img_path)
        X_df = pd.DataFrame([feats], columns=FEATURE_ORDER).astype(float)  # [FIX] 保留欄名避免 sklearn warning
        X_scaled = scaler.transform(X_df) if scaler is not None else X_df.values

        # 預測 offsets
        if model is not None:
            pred = model.predict(X_scaled, verbose=0)[0]
        else:
            pred = np.zeros(len(CLASSES)*4, dtype=float)

        # [NEW] 降暴：blend 回初始框，避免剛訓練完偏移過猛
        pred = pred * BLEND

        if PRINT_FIRST_PRED and (not printed_pred_once):
            print("Pred deltas (first row after blend):", pred[:min(12, len(pred))])
            printed_pred_once = True

        # 重組每個類別的 4 維 offset
        deltas = {cls: pred[i*4:(i+1)*4] for i, cls in enumerate(CLASSES)}
        final_boxes = {cls: apply_offset(INIT_BOXES[cls], deltas[cls], W, H) for cls in CLASSES}

        # 渲染
        canvas = template_rgba.copy()
        draw = ImageDraw.Draw(canvas)

        if DRAW_DEBUG_BOXES:
            for k,b in final_boxes.items():
                _stroke(draw, b)  # [NEW] 畫出預測框方便檢查是否飛出畫布

        # 圖片
        paste_image_into_box(canvas, prod_img_path, final_boxes["Product Image Position"])
        paste_image_into_box(canvas, icon_img_path,  final_boxes["Icon Position"])

        # 文字（黑色，Noto/蘋方）
        draw_text_in_box(draw, row.get(MAP_TEXT["Brand Position"], ""),          final_boxes["Brand Position"],          font_path=FONT_BOLD_PATH,   max_font=72)
        draw_text_in_box(draw, row.get(MAP_TEXT["Product Position"], ""),        final_boxes["Product Position"],        font_path=FONT_REGULAR_PATH, max_font=64)
        fab_text = str(row.get(MAP_TEXT["FAB Position"], "") or "").replace(";", "\n")
        draw_text_in_box(draw, fab_text,                                        final_boxes["FAB Position"],            font_path=FONT_REGULAR_PATH, max_font=44)
        draw_text_in_box(draw, row.get(MAP_TEXT["VIP Price Position"], ""),      final_boxes["VIP Price Position"],      font_path=FONT_BOLD_PATH,    max_font=100)
        draw_text_in_box(draw, row.get(MAP_TEXT["Non VIP Price Position"], ""),  final_boxes["Non VIP Price Position"],  font_path=FONT_REGULAR_PATH, max_font=36)
        draw_text_in_box(draw, row.get(MAP_TEXT["Original Price Position"], ""), final_boxes["Original Price Position"], font_path=FONT_REGULAR_PATH, max_font=32)
        draw_text_in_box(draw, row.get(MAP_TEXT["Sales Period Position"], ""),   final_boxes["Sales Period Position"],   font_path=FONT_REGULAR_PATH, max_font=28, align="right")

        out_path = OUTPUT_DIR / f"card_cn_{idx+1:03d}.png"
        canvas.convert("RGB").save(out_path, quality=95)
        print("Saved:", out_path)

if __name__ == "__main__":
    main()

Pred deltas (first row after blend): [ 0.9903074   0.8369056  -0.7444322  -0.91772836  0.2106682   0.8673276
 -0.99901277 -0.29051143  0.9815127  -0.9967992  -0.8292238   0.6552934 ]
Saved: out_cards_cn/card_cn_001.png
Saved: out_cards_cn/card_cn_002.png


In [100]:
# -*- coding: utf-8 -*-
import json, csv, re, argparse, sys, math
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
import pickle

from PIL import Image, ImageDraw, ImageFont
import textwrap
import unicodedata
import requests
from io import BytesIO  # [FIX] load_image_any 需要 BytesIO

# =================== 全域設定 ===================
# 模型/推論控制
USE_MODEL = True          # [NEW] 先關掉可做「零偏移」基準測試
BLEND = 1              # [NEW] 讓模型偏移更保守；零偏移時設 0
DRAW_DEBUG_BOXES = True   # [NEW] 畫出預測框方便對位除錯
PRINT_FIRST_PRED = True   # [NEW] 印第一筆的預測 delta

# 檔名/路徑
EXCEL_PATH = "Board Click SKU.xlsx"
SHEET_USER = "User_Input"
SHEET_IMG  = "SKU_Image"
SHEET_ICON = "SKU_Icon"

TEMPLATE_PATH = "sasa_pink_1280.png"
INITIAL_BOXES_PATH = "Initial_boxes.json"

MODEL_PATH = "layout_model.keras"
SCALER_PATH = "scaler.pkl"

OUT_CSV = "offsets_for_tf.csv"     # [NEW] 訓練資料輸出名（也用來找 .classes.json）
CLASS_ORDER_FILE = Path(OUT_CSV).with_suffix(".classes.json")  # [NEW]

OUTPUT_DIR = Path("out_cards_cn")
OUTPUT_DIR.mkdir(exist_ok=True)

# 欄位映射（Excel→語義）
MAP_TEXT = {
    "Brand Position":          "Brand",
    "VIP Price Position":      "VIP價",
    "Non VIP Price Position":  "優惠價",
    "Original Price Position": "建議價",
    "Product Position":        "Product name",
    "Sales Period Position":   "優惠期",
    "FAB Position":            "FAB",
}
COL_PRODUCT_SKU = "Product SKU"
COL_ICON_NAME   = "Icon"

# 圖片連結欄位
SKU_IMG_SKU_COL   = "Product SKU"
SKU_IMG_LINK_COL  = "Image"
SKU_ICON_NAME_COL = "Icon"
SKU_ICON_LINK_COL = "Icon Image"

# 字型（請指到你機器上存在的字型）
FONT_REGULAR_PATH = Path.home() / "Library/Fonts/NotoSansCJKsc-Regular.otf"
FONT_BOLD_PATH    = Path.home() / "Library/Fonts/NotoSansCJKsc-Bold.otf"

# =================== 工具函式 ===================
def safe_str(x): 
    return "" if (x is None or (isinstance(x, float) and pd.isna(x))) else str(x)

def count_digits(s): 
    return sum(ch.isdigit() for ch in safe_str(s))

def read_json_file(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    text = path.read_text(encoding="utf-8").strip()
    if not text:
        raise ValueError(f"File is empty: {path}")
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON in {path}") from e

def parse_ls_rect(item):
    v = item["value"]
    label = v["rectanglelabels"][0] if isinstance(v.get("rectanglelabels"), list) else v.get("labels", [""])[0]
    rect_pct = dict(x=v["x"], y=v["y"], w=v["width"], h=v["height"])
    ow = item.get("original_width") or item.get("image_original_width")
    oh = item.get("original_height") or item.get("image_original_height")
    if ow is None or oh is None:
        raise KeyError("Label Studio item missing original_width/original_height.")
    return label, rect_pct, int(ow), int(oh)

def pct_to_px(rpct, ow, oh):
    return {
        "x": rpct["x"] / 100.0 * ow,
        "y": rpct["y"] / 100.0 * oh,
        "w": rpct["w"] / 100.0 * ow,
        "h": rpct["h"] / 100.0 * oh,
    }

def resize_rect(rect, from_w, from_h, to_w, to_h):
    sx, sy = to_w / float(from_w), to_h / float(from_h)
    return {
        "x": rect["x"] * sx,
        "y": rect["y"] * sy,
        "w": rect["w"] * sx,
        "h": rect["h"] * sy,
    }

def delta_from(gt, init, W, H):
    w0 = max(1e-6, float(init["w"]))
    h0 = max(1e-6, float(init["h"]))
    return {
        "dx":   (float(gt["x"]) - float(init["x"])) / float(W),
        "dy":   (float(gt["y"]) - float(init["y"])) / float(H),
        "dlogw": float(np.log(max(1e-6, float(gt["w"])) / w0)),
        "dlogh": float(np.log(max(1e-6, float(gt["h"])) / h0)),
    }

def build_features(meta_row: dict):
    title  = safe_str(meta_row.get("title", ""))
    bullets = safe_str(meta_row.get("bullets", ""))
    vip    = safe_str(meta_row.get("vip_price", ""))
    unitp  = safe_str(meta_row.get("unit_price", ""))
    nonvip = safe_str(meta_row.get("non_vip_price", ""))
    has_badge = int(str(meta_row.get("has_badge", 0)).strip() not in ["", "0", "False", "false"])

    feats = {
        "title_len": len(title),
        "bullets_len": len(bullets),
        "bullets_lines": len([b for b in re.split(r"[;\n]", bullets) if b.strip()]),
        "vip_digits": count_digits(vip),
        "unit_digits": count_digits(unitp),
        "nonvip_digits": count_digits(nonvip),
        "has_badge": has_badge,
    }
    return feats

# =================== 產生訓練 CSV ===================
def run_preprocess(label_studio_json: Path, initial_boxes_json: Path, out_csv: Path, meta_csv: Path|None=None):
    init_cfg = read_json_file(initial_boxes_json)
    Tw = int(init_cfg["canvas"]["width"])
    Th = int(init_cfg["canvas"]["height"])
    init_boxes = init_cfg["boxes"]
    classes = list(init_boxes.keys())  # 訓練目標順序

    meta_df = None
    if meta_csv and Path(meta_csv).exists():
        meta_df = pd.read_csv(meta_csv)

    tasks = read_json_file(label_studio_json)

    rows = []
    for t in tasks:
        img_field = t.get("file_upload") or t.get("data", {}).get("image") or t.get("id")
        image_id = Path(str(img_field)).stem

        annos = t.get("annotations") or t.get("completions") or []
        if not annos:
            continue
        res = annos[0].get("result", [])

        gt_scaled = {}
        for r in res:
            if r.get("type") not in ("rectanglelabels", "rectangles"):
                continue
            label, rpct, ow, oh = parse_ls_rect(r)
            if label not in classes:
                continue
            rect_px = pct_to_px(rpct, ow, oh)
            rect_tpl = resize_rect(rect_px, from_w=ow, from_h=oh, to_w=Tw, to_h=Th)
            gt_scaled[label] = rect_tpl

        row = {"image_id": image_id}

        if meta_df is not None and "image_id" in meta_df.columns:
            m = meta_df.loc[meta_df["image_id"] == image_id]
            feats = build_features(m.iloc[0].to_dict()) if not m.empty else build_features({})
        else:
            feats = build_features({})
        row.update(feats)

        for cls in classes:
            init = init_boxes[cls]
            gt = gt_scaled.get(cls)
            d = {"dx": 0.0, "dy": 0.0, "dlogw": 0.0, "dlogh": 0.0} if gt is None else delta_from(gt, init, Tw, Th)
            row[f"{cls}_dx"]    = d["dx"]
            row[f"{cls}_dy"]    = d["dy"]
            row[f"{cls}_dlogw"] = d["dlogw"]
            row[f"{cls}_dlogh"] = d["dlogh"]

        rows.append(row)

    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False, quoting=csv.QUOTE_MINIMAL)
    print(f"[OK] Saved {out_csv}  rows={len(df)}")
    print("Columns:", list(df.columns))

    # 存「訓練時」的類別順序
    with open(Path(out_csv).with_suffix(".classes.json"), "w", encoding="utf-8") as f:
        json.dump(classes, f, ensure_ascii=False, indent=2)  # [NEW]
    print(f"[INFO] Saved class order -> {Path(out_csv).with_suffix('.classes.json')}")
    return df

# 命令列入口
def main_cli():
    ap = argparse.ArgumentParser()
    ap.add_argument("--label_studio_json", required=True)
    ap.add_argument("--initial_boxes_json", required=True)
    ap.add_argument("--out_csv", required=True)
    ap.add_argument("--meta_csv", default="")
    args = ap.parse_args()
    run_preprocess(
        label_studio_json=Path(args.label_studio_json),
        initial_boxes_json=Path(args.initial_boxes_json),
        out_csv=Path(args.out_csv),
        meta_csv=Path(args.meta_csv) if args.meta_csv else None
    )

if __name__ == "__main__" and not hasattr(sys.modules["__main__"], "__file__"):
    pass
elif __name__ == "__main__":
    main_cli()

# =================== （可選）重新產生訓練資料 ===================
df_FeatureEngineering = run_preprocess(
    label_studio_json=Path("Objects_positions.json"),
    initial_boxes_json=Path(INITIAL_BOXES_PATH),
    out_csv=Path(OUT_CSV),
    meta_csv=None
)

# ===== 建模 =====
df = pd.read_csv(OUT_CSV)
X = df.drop(columns=[c for c in df.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh")) or c=="image_id"])
Y = df[[c for c in df.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh"))]]

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(Y_train.shape[1])
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='mse',
              metrics=[MeanSquaredError(), MeanAbsoluteError()])

history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=50,
    batch_size=16
)

# 簡單 sanity check
_ = model.predict(X_val[0:1])
model.save(MODEL_PATH)
with open(SCALER_PATH, "wb") as f:
    pickle.dump(scaler, f)
print("Model & scaler saved.")

# =================== 產圖所需常用工具 ===================
def _load_font(path, size):
    return ImageFont.truetype(str(path), size)

def _safe(s): return "" if (s is None or (isinstance(s, float) and pd.isna(s))) else str(s)
def _count_digits(s): return sum(ch.isdigit() for ch in _safe(s))

def _norm_key(x):
    s = "" if pd.isna(x) else str(x)
    s = unicodedata.normalize("NFKC", s)
    return s.strip().casefold()

def _drive_share_to_direct(u: str) -> str:
    if not u: return u
    m = re.search(r"/d/([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    m = re.search(r"[?&]id=([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    return u

def load_image_any(path_or_url: str):
    if not path_or_url:
        return None
    s = str(path_or_url).strip()
    try:
        if s.startswith("http://") or s.startswith("https://"):
            s2 = _drive_share_to_direct(s)
            resp = requests.get(s2, timeout=15)
            resp.raise_for_status()
            return Image.open(BytesIO(resp.content)).convert("RGBA")
        p = Path(s)
        if p.exists():
            return Image.open(p).convert("RGBA")
    except Exception as e:
        print("[WARN] cannot load image:", s, e)
    return None

FEATURE_ORDER = [
    "title_len", "bullets_len", "bullets_lines",
    "vip_digits", "unit_digits", "nonvip_digits",
    "has_badge",
]

def build_features_from_row(row: pd.Series, resolved_icon_path: str) -> dict:
    title   = _safe(row.get(MAP_TEXT["Product Position"], ""))
    bullets = _safe(row.get(MAP_TEXT["FAB Position"], ""))
    vip     = _safe(row.get(MAP_TEXT["VIP Price Position"], ""))
    unitp   = _safe(row.get(MAP_TEXT["Non VIP Price Position"], ""))
    orig    = _safe(row.get(MAP_TEXT["Original Price Position"], ""))
    has_icon = int(Path(_safe(resolved_icon_path)).exists()) if resolved_icon_path else 0

    return {
        "title_len": len(title),
        "bullets_len": len(bullets),
        "bullets_lines": len([b for b in bullets.split(";") if b.strip()]),
        "vip_digits": _count_digits(vip),
        "unit_digits": _count_digits(unitp),
        "nonvip_digits": _count_digits(unitp),  # [FIX] 與訓練時一致（non_vip_price）
        "has_badge": has_icon,
    }

def _clip(v, lo, hi):  # [NEW]
    return max(lo, min(hi, v))

def apply_offset(init_box, delta, W, H):
    dx, dy, dlogw, dlogh = map(float, delta)

    # [NEW] 位移±0.5 畫布；可按資料調整
    dx = _clip(dx, -0.5, 0.5)
    dy = _clip(dy, -0.5, 0.5)

    # [NEW] 尺寸限制 0.5x~2x
    sx = math.exp(_clip(dlogw, math.log(0.5), math.log(2.0)))
    sy = math.exp(_clip(dlogh, math.log(0.5), math.log(2.0)))

    x = init_box["x"] + dx * W
    y = init_box["y"] + dy * H
    w = max(24, init_box["w"] * sx)   # [NEW] 最小寬高
    h = max(24, init_box["h"] * sy)

    # [NEW] 裁進畫布
    x = _clip(x, 0, W - w)
    y = _clip(y, 0, H - h)

    return {"x": x, "y": y, "w": w, "h": h}

def paste_image_into_box(canvas_rgba, path_or_url, box, padding=6):
    im = load_image_any(path_or_url)
    if im is None:
        print("[WARN] image not found:", path_or_url)
        return
    x, y, w, h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    w2, h2 = max(1, w - padding*2), max(1, h - padding*2)
    ratio = min(w2 / im.width, h2 / im.height)
    im = im.resize((max(1,int(im.width*ratio)), max(1,int(im.height*ratio))), Image.LANCZOS)
    ox = x + (w - im.width)//2
    oy = y + (h - im.height)//2
    canvas_rgba.alpha_composite(im, (ox, oy))

def draw_text_in_box(draw, text, box, font_path, max_font=64, min_font=16, align="left", line_spacing=1.15):
    if not text or str(text).strip()=="":
        return
    x, y, w, h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    text = str(text)

    for fs in range(max_font, min_font-1, -2):
        font = _load_font(font_path, fs)
        approx_chars = max(1, int(w / (fs * 0.55)))
        lines = []
        for raw in text.split("\n"):
            lines += (textwrap.wrap(raw, width=approx_chars) if approx_chars>1 else [raw])

        bboxes = [draw.textbbox((0,0), ln, font=font) for ln in lines]
        line_heights = [bb[3]-bb[1] for bb in bboxes]
        total_h = int(sum(line_heights) + (len(lines)-1)*fs*(line_spacing-1))
        if total_h <= h:
            cur_y = y + (h - total_h)//2
            for ln in lines:
                bb = draw.textbbox((0,0), ln, font=font)
                lw = bb[2]-bb[0]
                if align == "center":
                    cur_x = x + (w - lw)//2
                elif align == "right":
                    cur_x = x + (w - lw)
                else:
                    cur_x = x
                draw.text((cur_x, cur_y), ln, font=font, fill=(0,0,0,255))
                cur_y += int(fs * line_spacing)
            return
    draw.text((x, y), text.split("\n")[0][:30]+"…", font=_load_font(font_path, min_font), fill=(0,0,0,255))

def _stroke(draw, box, color=(0,0,0,255), width=2):  # [NEW]
    x,y,w,h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    draw.rectangle([x,y,x+w,y+h], outline=color, width=width)

def load_template_and_boxes():
    tpl = Image.open(TEMPLATE_PATH).convert("RGBA")
    tw, th = tpl.size
    init_cfg = json.loads(Path(INITIAL_BOXES_PATH).read_text(encoding="utf-8"))
    cw, ch = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    if (tw, th) != (cw, ch):
        sx, sy = tw/float(cw), th/float(ch)
        for _, r in init_cfg["boxes"].items():
            r["x"], r["y"] = r["x"]*sx, r["y"]*sy
            r["w"], r["h"] = r["w"]*sx, r["h"]*sy
        init_cfg["canvas"]["width"], init_cfg["canvas"]["height"] = tw, th
        print(f"[INFO] Scaled initial boxes to {tw}x{th}")
    return tpl, init_cfg

def load_model_and_scaler():
    model = tf.keras.models.load_model(MODEL_PATH) if Path(MODEL_PATH).exists() else None
    scaler = pickle.load(open(SCALER_PATH, "rb")) if Path(SCALER_PATH).exists() else None
    if model is None:  print("[WARN] MODEL not found -> zero offsets")
    if scaler is None: print("[WARN] SCALER not found -> use raw features")
    return model, scaler

# [NEW] 把目前框存成新的 baseline（可選）
def save_as_new_initial_boxes(path, canvas_size, boxes_dict):
    data = {"canvas": {"width": canvas_size[0], "height": canvas_size[1]}, "boxes": boxes_dict}
    Path(path).write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")
    print("Saved new Initial_boxes to:", path)

# =================== 產圖主流程 ===================
def main():
    # 讀 Excel
    xls = pd.ExcelFile(EXCEL_PATH)
    df_user = pd.read_excel(xls, sheet_name=SHEET_USER)
    df_img  = pd.read_excel(xls, sheet_name=SHEET_IMG)
    df_icon = pd.read_excel(xls, sheet_name=SHEET_ICON)

    img_lookup  = {_norm_key(r[SKU_IMG_SKU_COL]): str(r[SKU_IMG_LINK_COL]).strip()
                   for _, r in df_img.iterrows() if SKU_IMG_SKU_COL in r and SKU_IMG_LINK_COL in r}
    icon_lookup = {_norm_key(r[SKU_ICON_NAME_COL]): str(r[SKU_ICON_LINK_COL]).strip()
                   for _, r in df_icon.iterrows() if SKU_ICON_NAME_COL in r and SKU_ICON_LINK_COL in r}

    template_rgba, init_cfg = load_template_and_boxes()
    W, H = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    INIT_BOXES = init_cfg["boxes"]

    # 讀回訓練時的類別順序（避免對錯框）
    if CLASS_ORDER_FILE.exists():
        train_classes = json.loads(CLASS_ORDER_FILE.read_text(encoding="utf-8"))
        if set(train_classes) != set(INIT_BOXES.keys()):
            raise ValueError(f"Classes mismatch.\nTrain:{train_classes}\nInfer:{list(INIT_BOXES.keys())}")
        CLASSES = train_classes[:]  # [NEW]
    else:
        CLASSES = list(INIT_BOXES.keys())

    model, scaler = load_model_and_scaler()
    printed_pred_once = False

    for idx, row in df_user.iterrows():
        sku_key  = _norm_key(row.get(COL_PRODUCT_SKU, ""))
        icon_key = _norm_key(row.get(COL_ICON_NAME, ""))

        prod_img_path = img_lookup.get(sku_key, "")
        icon_img_path = icon_lookup.get(icon_key, "")

        # ==== 特徵 & 預測 ====
        if USE_MODEL and (model is not None) and (scaler is not None):
            feats = build_features_from_row(row, resolved_icon_path=icon_img_path)
            X_df = pd.DataFrame([feats], columns=FEATURE_ORDER).astype(float)   # [FIX] DataFrame 保欄名
            X_scaled = scaler.transform(X_df)
            pred = model.predict(X_scaled, verbose=0)[0]
            pred = pred * BLEND                                                # [NEW] 降暴
            if PRINT_FIRST_PRED and (not printed_pred_once):
                print("Pred deltas (first row, after BLEND):", pred[:min(12, len(pred))])
                printed_pred_once = True
            deltas = {cls: pred[i*4:(i+1)*4] for i, cls in enumerate(CLASSES)}
        else:
            # 零偏移：用 baseline 直接渲染（做對位基準檢查）
            deltas = {cls: np.array([0.0, 0.0, 0.0, 0.0]) for cls in CLASSES}  # [NEW]

        final_boxes = {cls: apply_offset(INIT_BOXES[cls], deltas[cls], W, H) for cls in CLASSES}

        # ==== 繪圖 ====
        canvas = template_rgba.copy()
        draw = ImageDraw.Draw(canvas)

        if DRAW_DEBUG_BOXES:
            for k,b in final_boxes.items():
                _stroke(draw, b)

        paste_image_into_box(canvas, prod_img_path, final_boxes["Product Image Position"])
        paste_image_into_box(canvas, icon_img_path,  final_boxes["Icon Position"])

        draw_text_in_box(draw, row.get(MAP_TEXT["Brand Position"], ""),          final_boxes["Brand Position"],          font_path=FONT_BOLD_PATH,   max_font=72)
        draw_text_in_box(draw, row.get(MAP_TEXT["Product Position"], ""),        final_boxes["Product Position"],        font_path=FONT_REGULAR_PATH, max_font=64)
        fab_text = str(row.get(MAP_TEXT["FAB Position"], "") or "").replace(";", "\n")
        draw_text_in_box(draw, fab_text,                                        final_boxes["FAB Position"],            font_path=FONT_REGULAR_PATH, max_font=44)
        draw_text_in_box(draw, row.get(MAP_TEXT["VIP Price Position"], ""),      final_boxes["VIP Price Position"],      font_path=FONT_BOLD_PATH,    max_font=100)
        draw_text_in_box(draw, row.get(MAP_TEXT["Non VIP Price Position"], ""),  final_boxes["Non VIP Price Position"],  font_path=FONT_REGULAR_PATH, max_font=36)
        draw_text_in_box(draw, row.get(MAP_TEXT["Original Price Position"], ""), final_boxes["Original Price Position"], font_path=FONT_REGULAR_PATH, max_font=32)
        draw_text_in_box(draw, row.get(MAP_TEXT["Sales Period Position"], ""),   final_boxes["Sales Period Position"],   font_path=FONT_REGULAR_PATH, max_font=28, align="right")

        out_path = OUTPUT_DIR / f"card_cn_{idx+1:03d}.png"
        canvas.convert("RGB").save(out_path, quality=95)
        print("Saved:", out_path)

    # 想把目前的 final_boxes 存成新的 baseline（人工調好後再解註）：
    # save_as_new_initial_boxes(INITIAL_BOXES_PATH, (W, H), final_boxes)  # [NEW]

if __name__ == "__main__":
    main()


[OK] Saved offsets_for_tf.csv  rows=29
Columns: ['image_id', 'title_len', 'bullets_len', 'bullets_lines', 'vip_digits', 'unit_digits', 'nonvip_digits', 'has_badge', 'Sales Period Position_dx', 'Sales Period Position_dy', 'Sales Period Position_dlogw', 'Sales Period Position_dlogh', 'Brand Position_dx', 'Brand Position_dy', 'Brand Position_dlogw', 'Brand Position_dlogh', 'Product Position_dx', 'Product Position_dy', 'Product Position_dlogw', 'Product Position_dlogh', 'FAB Position_dx', 'FAB Position_dy', 'FAB Position_dlogw', 'FAB Position_dlogh', 'Product Image Position_dx', 'Product Image Position_dy', 'Product Image Position_dlogw', 'Product Image Position_dlogh', 'VIP Price Position_dx', 'VIP Price Position_dy', 'VIP Price Position_dlogw', 'VIP Price Position_dlogh', 'Non VIP Price Position_dx', 'Non VIP Price Position_dy', 'Non VIP Price Position_dlogw', 'Non VIP Price Position_dlogh', 'Original Price Position_dx', 'Original Price Position_dy', 'Original Price Position_dlogw', 'Ori

/opt/anaconda3/envs/boardclick/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - loss: 0.0680 - mean_absolute_error: 0.1592 - mean_squared_error: 0.0680 - val_loss: 0.0647 - val_mean_absolute_error: 0.1542 - val_mean_squared_error: 0.0647
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0675 - mean_absolute_error: 0.1581 - mean_squared_error: 0.0675 - val_loss: 0.0643 - val_mean_absolute_error: 0.1535 - val_mean_squared_error: 0.0643
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0670 - mean_absolute_error: 0.1571 - mean_squared_error: 0.0670 - val_loss: 0.0639 - val_mean_absolute_error: 0.1528 - val_mean_squared_error: 0.0639
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0665 - mean_absolute_error: 0.1561 - mean_squared_error: 0.0665 - val_loss: 0.0634 - val_mean_absolute_error: 0.1523 - val_mean_squared_error: 0.0634
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0660 - mean_absolute_error: 0.1552 - mean_squared_error: 0.0660 - val_loss: 0.0630 - val_mean_absolute_error: 0.151

In [125]:
# -*- coding: utf-8 -*-
"""
完整版：Label Studio → offsets_for_tf.csv → 訓練(tanh+label scaling) → 版面推論與出圖
已含：
- [FIX] nonvip_digits 特徵與訓練一致
- [FIX] sklearn warning：推論時用 DataFrame 給 scaler.transform()
- [NEW] 類別順序 *.classes.json 存取，避免訓練/推論順序不一致
- [NEW] 模型輸出層 tanh + 標籤縮放（dx,dy=0.3；dlog=0.4）
- [NEW] USE_MODEL/BLEND 開關、有描框除錯
- [NEW] 邊界保護：位移/縮放/最小尺寸/裁進畫布
"""

import json, csv, re, argparse, sys, math
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
import pickle

from PIL import Image, ImageDraw, ImageFont
import textwrap
import unicodedata
import requests
from io import BytesIO  # [FIX] 給 load_image_any 使用

# =================== 全域設定 ===================
# 模型/推論控制
USE_MODEL = True        # [NEW] 先關掉可做「零偏移」基準測試
BLEND = 0.3             # [NEW] 讓模型偏移更保守；零偏移時設 0
DRAW_DEBUG_BOXES = True   # [NEW] 畫出預測框方便對位除錯
PRINT_FIRST_PRED = True   # [NEW] 印第一筆的預測 delta

# Label scaling（對應 tanh 輸出）
DX_DY_SCALE = 0.3         # [NEW] dx,dy 的尺度
DLOG_SCALE  = 0.4         # [NEW] dlogw,dlogh 的尺度

# 檔名/路徑
EXCEL_PATH = "Board Click SKU.xlsx"
SHEET_USER = "User_Input"
SHEET_IMG  = "SKU_Image"
SHEET_ICON = "SKU_Icon"

TEMPLATE_PATH = "sasa_pink_1280.png"
INITIAL_BOXES_PATH = "Initial_boxes.json"

MODEL_PATH = "layout_model.keras"
SCALER_PATH = "scaler.pkl"

OUT_CSV = "offsets_for_tf.csv"     # 訓練資料輸出名（也用來找 .classes.json）
CLASS_ORDER_FILE = Path(OUT_CSV).with_suffix(".classes.json")  # [NEW]

OUTPUT_DIR = Path("out_cards_cn")
OUTPUT_DIR.mkdir(exist_ok=True)

# 欄位映射（Excel→語義）
MAP_TEXT = {
    "Brand Position":          "Brand",
    "VIP Price Position":      "VIP價",
    "Non VIP Price Position":  "優惠價",
    "Original Price Position": "建議價",
    "Product Position":        "Product name",
    "Sales Period Position":   "優惠期",
    "FAB Position":            "FAB",
}
COL_PRODUCT_SKU = "Product SKU"
COL_ICON_NAME   = "Icon"

# 圖片連結欄位
SKU_IMG_SKU_COL   = "Product SKU"
SKU_IMG_LINK_COL  = "Image"
SKU_ICON_NAME_COL = "Icon"
SKU_ICON_LINK_COL = "Icon Image"

# 字型（請指到你機器上存在的字型）
FONT_REGULAR_PATH = Path.home() / "Library/Fonts/NotoSansCJKsc-Regular.otf"
FONT_BOLD_PATH    = Path.home() / "Library/Fonts/NotoSansCJKsc-Bold.otf"

# =================== 共用小工具 ===================
def safe_str(x): 
    return "" if (x is None or (isinstance(x, float) and pd.isna(x))) else str(x)

def count_digits(s): 
    return sum(ch.isdigit() for ch in safe_str(s))

def read_json_file(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    text = path.read_text(encoding="utf-8").strip()
    if not text:
        raise ValueError(f"File is empty: {path}")
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        preview = text[:200]
        raise ValueError(f"Invalid JSON in {path} (preview: {preview!r})") from e

def parse_ls_rect(item):
    v = item["value"]
    label = v["rectanglelabels"][0] if isinstance(v.get("rectanglelabels"), list) else v.get("labels", [""])[0]
    rect_pct = dict(x=v["x"], y=v["y"], w=v["width"], h=v["height"])
    ow = item.get("original_width") or item.get("image_original_width")
    oh = item.get("original_height") or item.get("image_original_height")
    if ow is None or oh is None:
        raise KeyError("Label Studio item missing original_width/original_height.")
    return label, rect_pct, int(ow), int(oh)

def pct_to_px(rpct, ow, oh):
    return {"x": rpct["x"]/100.0*ow, "y": rpct["y"]/100.0*oh, "w": rpct["w"]/100.0*ow, "h": rpct["h"]/100.0*oh}

def resize_rect(rect, from_w, from_h, to_w, to_h):
    sx, sy = to_w/float(from_w), to_h/float(from_h)
    return {"x": rect["x"]*sx, "y": rect["y"]*sy, "w": rect["w"]*sx, "h": rect["h"]*sy}

def delta_from(gt, init, W, H):
    w0 = max(1e-6, float(init["w"]))
    h0 = max(1e-6, float(init["h"]))
    return {
        "dx":   (float(gt["x"]) - float(init["x"])) / float(W),
        "dy":   (float(gt["y"]) - float(init["y"])) / float(H),
        "dlogw": float(np.log(max(1e-6, float(gt["w"])) / w0)),
        "dlogh": float(np.log(max(1e-6, float(gt["h"])) / h0)),
    }

def _load_font(path, size): return ImageFont.truetype(str(path), size)
def _safe(s): return "" if (s is None or (isinstance(s, float) and pd.isna(s))) else str(s)
def _count_digits(s): return sum(ch.isdigit() for ch in _safe(s))

def _norm_key(x):
    s = "" if pd.isna(x) else str(x)
    s = unicodedata.normalize("NFKC", s)
    return s.strip().casefold()

def _drive_share_to_direct(u: str) -> str:
    if not u: return u
    m = re.search(r"/d/([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    m = re.search(r"[?&]id=([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    return u

def load_image_any(path_or_url: str):
    if not path_or_url:
        return None
    s = str(path_or_url).strip()
    try:
        if s.startswith("http://") or s.startswith("https://"):
            s2 = _drive_share_to_direct(s)
            resp = requests.get(s2, timeout=15)
            resp.raise_for_status()
            return Image.open(BytesIO(resp.content)).convert("RGBA")
        p = Path(s)
        if p.exists():
            return Image.open(p).convert("RGBA")
    except Exception as e:
        print("[WARN] cannot load image:", s, e)
    return None

def paste_image_into_box(canvas_rgba, path_or_url, box, padding=6):
    im = load_image_any(path_or_url)
    if im is None:
        print("[WARN] image not found:", path_or_url)
        return
    x, y, w, h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    w2, h2 = max(1, w - padding*2), max(1, h - padding*2)
    ratio = min(w2 / im.width, h2 / im.height)
    im = im.resize((max(1,int(im.width*ratio)), max(1,int(im.height*ratio))), Image.LANCZOS)
    ox = x + (w - im.width)//2
    oy = y + (h - im.height)//2
    canvas_rgba.alpha_composite(im, (ox, oy))

def draw_text_in_box(draw, text, box, font_path, max_font=64, min_font=16, align="left", line_spacing=1.15):
    if not text or str(text).strip()=="":
        return
    x, y, w, h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    text = str(text)

    for fs in range(max_font, min_font-1, -2):
        font = _load_font(font_path, fs)
        approx_chars = max(1, int(w / (fs * 0.55)))
        lines = []
        for raw in text.split("\n"):
            lines += (textwrap.wrap(raw, width=approx_chars) if approx_chars>1 else [raw])

        bboxes = [draw.textbbox((0,0), ln, font=font) for ln in lines]
        line_heights = [bb[3]-bb[1] for bb in bboxes]
        total_h = int(sum(line_heights) + (len(lines)-1)*fs*(line_spacing-1))
        if total_h <= h:
            cur_y = y + (h - total_h)//2
            for ln in lines:
                bb = draw.textbbox((0,0), ln, font=font)
                lw = bb[2]-bb[0]
                if align == "center":
                    cur_x = x + (w - lw)//2
                elif align == "right":
                    cur_x = x + (w - lw)
                else:
                    cur_x = x
                draw.text((cur_x, cur_y), ln, font=font, fill=(0,0,0,255))
                cur_y += int(fs * line_spacing)
            return
    draw.text((x, y), text.split("\n")[0][:30]+"…", font=_load_font(font_path, min_font), fill=(0,0,0,255))

def _stroke(draw, box, color=(0,0,0,255), width=2):  # [NEW] 偵錯描框
    x,y,w,h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    draw.rectangle([x,y,x+w,y+h], outline=color, width=width)

def _clip(v, lo, hi):  # [NEW]
    return max(lo, min(hi, v))

# =================== 特徵工程（與訓練一致） ===================
FEATURE_ORDER = [
    "title_len", "bullets_len", "bullets_lines",
    "vip_digits", "unit_digits", "nonvip_digits",
    "has_badge",
]

def build_features(meta_row: dict):
    title  = safe_str(meta_row.get("title", ""))
    bullets = safe_str(meta_row.get("bullets", ""))
    vip    = safe_str(meta_row.get("vip_price", ""))
    unitp  = safe_str(meta_row.get("unit_price", ""))
    nonvip = safe_str(meta_row.get("non_vip_price", ""))
    has_badge = int(str(meta_row.get("has_badge", 0)).strip() not in ["", "0", "False", "false"])

    feats = {
        "title_len": len(title),
        "bullets_len": len(bullets),
        "bullets_lines": len([b for b in re.split(r"[;\n]", bullets) if b.strip()]),
        "vip_digits": count_digits(vip),
        "unit_digits": count_digits(unitp),
        "nonvip_digits": count_digits(nonvip),
        "has_badge": has_badge,
    }
    return feats

def build_features_from_row(row: pd.Series, resolved_icon_path: str) -> dict:
    title   = _safe(row.get(MAP_TEXT["Product Position"], ""))
    bullets = _safe(row.get(MAP_TEXT["FAB Position"], ""))
    vip     = _safe(row.get(MAP_TEXT["VIP Price Position"], ""))
    unitp   = _safe(row.get(MAP_TEXT["Non VIP Price Position"], ""))
    orig    = _safe(row.get(MAP_TEXT["Original Price Position"], ""))
    has_icon = int(Path(_safe(resolved_icon_path)).exists()) if resolved_icon_path else 0

    return {
        "title_len": len(title),
        "bullets_len": len(bullets),
        "bullets_lines": len([b for b in bullets.split(";") if b.strip()]),
        "vip_digits": _count_digits(vip),
        "unit_digits": _count_digits(unitp),
        "nonvip_digits": _count_digits(unitp),  # [FIX] 與訓練一致（non_vip_price）
        "has_badge": has_icon,
    }

# =================== 產生訓練 CSV ===================
def run_preprocess(label_studio_json: Path, initial_boxes_json: Path, out_csv: Path, meta_csv: Path|None=None):
    init_cfg = read_json_file(initial_boxes_json)
    Tw = int(init_cfg["canvas"]["width"])
    Th = int(init_cfg["canvas"]["height"])
    init_boxes = init_cfg["boxes"]
    classes = list(init_boxes.keys())

    meta_df = None
    if meta_csv and Path(meta_csv).exists():
        meta_df = pd.read_csv(meta_csv)

    tasks = read_json_file(label_studio_json)

    rows = []
    for t in tasks:
        img_field = t.get("file_upload") or t.get("data", {}).get("image") or t.get("id")
        image_id = Path(str(img_field)).stem

        annos = t.get("annotations") or t.get("completions") or []
        if not annos: continue
        res = annos[0].get("result", [])

        gt_scaled = {}
        for r in res:
            if r.get("type") not in ("rectanglelabels", "rectangles"): continue
            label, rpct, ow, oh = parse_ls_rect(r)
            if label not in classes: continue
            rect_px = pct_to_px(rpct, ow, oh)
            rect_tpl = resize_rect(rect_px, from_w=ow, from_h=oh, to_w=Tw, to_h=Th)
            gt_scaled[label] = rect_tpl

        row = {"image_id": image_id}

        if meta_df is not None and "image_id" in meta_df.columns:
            m = meta_df.loc[meta_df["image_id"] == image_id]
            feats = build_features(m.iloc[0].to_dict()) if not m.empty else build_features({})
        else:
            feats = build_features({})
        row.update(feats)

        for cls in classes:
            init = init_boxes[cls]
            gt = gt_scaled.get(cls)
            if gt is None:
                d = {"dx": 0.0, "dy": 0.0, "dlogw": 0.0, "dlogh": 0.0}
            else:
                d = delta_from(gt, init, Tw, Th)
            row[f"{cls}_dx"]    = d["dx"]
            row[f"{cls}_dy"]    = d["dy"]
            row[f"{cls}_dlogw"] = d["dlogw"]
            row[f"{cls}_dlogh"] = d["dlogh"]

        rows.append(row)

    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False, quoting=csv.QUOTE_MINIMAL)
    print(f"[OK] Saved {out_csv}  rows={len(df)}")
    print("Columns:", list(df.columns))

    # [NEW] 存「訓練時」的類別順序
    with open(Path(out_csv).with_suffix(".classes.json"), "w", encoding="utf-8") as f:
        json.dump(classes, f, ensure_ascii=False, indent=2)
    print(f"[INFO] Saved class order -> {Path(out_csv).with_suffix('.classes.json')}")
    return df

# =================== 模型訓練（tanh + 標籤縮放） ===================
def train_model():
    df = pd.read_csv(OUT_CSV)

    X = df.drop(columns=[c for c in df.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh")) or c=="image_id"])
    Y = df[[c for c in df.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh"))]]

    # [NEW] Label scaling：把標籤壓到 ~[-1,1]
    Y_scaled = Y.copy()
    for c in Y.columns:
        if c.endswith(("_dx", "_dy")):
            Y_scaled[c] = Y[c] / DX_DY_SCALE
        else:
            Y_scaled[c] = Y[c] / DLOG_SCALE

    X_train, X_val, Y_train, Y_val = train_test_split(X, Y_scaled, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(Y_train.shape[1], activation='tanh')  # [NEW] 限制輸出在 [-1,1]
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss='mse',
        metrics=[MeanSquaredError(), MeanAbsoluteError()]
    )

    model.fit(
        X_train, Y_train,
        validation_data=(X_val, Y_val),
        epochs=50,
        batch_size=16,
        verbose=1
    )

    model.save(MODEL_PATH)
    with open(SCALER_PATH, "wb") as f:
        pickle.dump(scaler, f)

    print("[OK] Model & scaler saved.")

# =================== 模板與初始盒 ===================
def load_template_and_boxes():
    tpl = Image.open(TEMPLATE_PATH).convert("RGBA")
    tw, th = tpl.size
    init_cfg = json.loads(Path(INITIAL_BOXES_PATH).read_text(encoding="utf-8"))
    cw, ch = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    if (tw, th) != (cw, ch):
        sx, sy = tw/float(cw), th/float(ch)
        for _, r in init_cfg["boxes"].items():
            r["x"], r["y"] = r["x"]*sx, r["y"]*sy
            r["w"], r["h"] = r["w"]*sx, r["h"]*sy
        init_cfg["canvas"]["width"], init_cfg["canvas"]["height"] = tw, th
        print(f"[INFO] Scaled initial boxes to {tw}x{th}")
    return tpl, init_cfg

def load_model_and_scaler():
    model = tf.keras.models.load_model(MODEL_PATH) if Path(MODEL_PATH).exists() else None
    scaler = pickle.load(open(SCALER_PATH, "rb")) if Path(SCALER_PATH).exists() else None
    if model is None:  print("[WARN] MODEL not found → zero offsets")
    if scaler is None: print("[WARN] SCALER not found → use raw features")
    return model, scaler

# =================== 推論（還原縮放） ===================
def predict_offsets(model, scaler, feats: dict, classes: list) -> np.ndarray:
    # [FIX] DataFrame + 指定欄名，避免 sklearn warning
    X_df = pd.DataFrame([feats], columns=FEATURE_ORDER).astype(float)
    X_scaled = scaler.transform(X_df) if scaler is not None else X_df.values
    if model is not None:
        pred_scaled = model.predict(X_scaled, verbose=0)[0]
    else:
        pred_scaled = np.zeros(len(classes)*4, dtype=float)

    # 還原尺度
    restored = []
    for i in range(len(classes)):
        dx = pred_scaled[i*4+0] * DX_DY_SCALE
        dy = pred_scaled[i*4+1] * DX_DY_SCALE
        dw = pred_scaled[i*4+2] * DLOG_SCALE
        dh = pred_scaled[i*4+3] * DLOG_SCALE
        restored.extend([dx, dy, dw, dh])
    return np.array(restored, dtype=float)

# =================== 幾何 / 佈局 ===================
def apply_offset(init_box, delta, W, H):
    dx, dy, dlogw, dlogh = map(float, delta)

    # [NEW] 位移限制（避免飛出半張圖；可依據資料調整）
    dx = _clip(dx, -0.5, 0.5)
    dy = _clip(dy, -0.5, 0.5)

    # [NEW] 尺寸縮放限制（0.5x ~ 2x）
    sx = math.exp(_clip(dlogw, math.log(0.5), math.log(2.0)))
    sy = math.exp(_clip(dlogh, math.log(0.5), math.log(2.0)))

    x = init_box["x"] + dx * W
    y = init_box["y"] + dy * H
    w = max(24, init_box["w"] * sx)  # 最小寬/高
    h = max(24, init_box["h"] * sy)

    # [NEW] 裁進畫布
    x = _clip(x, 0, W - w)
    y = _clip(y, 0, H - h)

    return {"x": x, "y": y, "w": w, "h": h}

# =================== 主流程（推論 & 畫圖） ===================
def main_render():
    # 讀 Excel
    xls = pd.ExcelFile(EXCEL_PATH)
    df_user = pd.read_excel(xls, sheet_name=SHEET_USER)
    df_img  = pd.read_excel(xls, sheet_name=SHEET_IMG)
    df_icon = pd.read_excel(xls, sheet_name=SHEET_ICON)

    img_lookup  = {_norm_key(r[SKU_IMG_SKU_COL]): str(r[SKU_IMG_LINK_COL]).strip()
                   for _, r in df_img.iterrows() if SKU_IMG_SKU_COL in r and SKU_IMG_LINK_COL in r}
    icon_lookup = {_norm_key(r[SKU_ICON_NAME_COL]): str(r[SKU_ICON_LINK_COL]).strip()
                   for _, r in df_icon.iterrows() if SKU_ICON_NAME_COL in r and SKU_ICON_LINK_COL in r}

    template_rgba, init_cfg = load_template_and_boxes()
    W, H = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    INIT_BOXES = init_cfg["boxes"]

    # [NEW] 讀回訓練時的類別順序（避免對錯框）
    if CLASS_ORDER_FILE.exists():
        train_classes = json.loads(CLASS_ORDER_FILE.read_text(encoding="utf-8"))
        if set(train_classes) != set(INIT_BOXES.keys()):
            raise ValueError(f"Classes mismatch.\nTrain:{train_classes}\nInfer:{list(INIT_BOXES.keys())}")
        CLASSES = train_classes[:]
    else:
        CLASSES = list(INIT_BOXES.keys())

    model, scaler = load_model_and_scaler()
    printed_pred_once = False

    for idx, row in df_user.iterrows():
        sku_key  = _norm_key(row.get(COL_PRODUCT_SKU, ""))
        icon_key = _norm_key(row.get(COL_ICON_NAME, ""))

        prod_img_path = img_lookup.get(sku_key, "")
        icon_img_path = icon_lookup.get(icon_key, "")

        # 特徵
        feats = build_features_from_row(row, resolved_icon_path=icon_img_path)

        # 預測 offsets（還原尺度）
        if USE_MODEL and (model is not None) and (scaler is not None):
            pred = predict_offsets(model, scaler, feats, CLASSES)
            pred = pred * BLEND  # [NEW] 降暴（可逐步拉到 1.0）
            if PRINT_FIRST_PRED and (not printed_pred_once):
                print("Pred deltas (first row, after BLEND):", pred[:min(12, len(pred))])
                printed_pred_once = True
        else:
            pred = np.zeros(len(CLASSES)*4, dtype=float)

        # 轉成 per-class 的 delta
        deltas = {cls: pred[i*4:(i+1)*4] for i, cls in enumerate(CLASSES)}
        final_boxes = {cls: apply_offset(INIT_BOXES[cls], deltas[cls], W, H) for cls in CLASSES}

        # 繪圖
        canvas = template_rgba.copy()
        draw = ImageDraw.Draw(canvas)

        if DRAW_DEBUG_BOXES:
            for k,b in final_boxes.items():
                _stroke(draw, b)

        paste_image_into_box(canvas, prod_img_path, final_boxes["Product Image Position"])
        paste_image_into_box(canvas, icon_img_path,  final_boxes["Icon Position"])

        draw_text_in_box(draw, row.get(MAP_TEXT["Brand Position"], ""),          final_boxes["Brand Position"],          font_path=FONT_BOLD_PATH,   max_font=72)
        draw_text_in_box(draw, row.get(MAP_TEXT["Product Position"], ""),        final_boxes["Product Position"],        font_path=FONT_REGULAR_PATH, max_font=64)
        fab_text = str(row.get(MAP_TEXT["FAB Position"], "") or "").replace(";", "\n")
        draw_text_in_box(draw, fab_text,                                        final_boxes["FAB Position"],            font_path=FONT_REGULAR_PATH, max_font=44)
        draw_text_in_box(draw, row.get(MAP_TEXT["VIP Price Position"], ""),      final_boxes["VIP Price Position"],      font_path=FONT_BOLD_PATH,    max_font=100)
        draw_text_in_box(draw, row.get(MAP_TEXT["Non VIP Price Position"], ""),  final_boxes["Non VIP Price Position"],  font_path=FONT_REGULAR_PATH, max_font=36)
        draw_text_in_box(draw, row.get(MAP_TEXT["Original Price Position"], ""), final_boxes["Original Price Position"], font_path=FONT_REGULAR_PATH, max_font=32)
        draw_text_in_box(draw, row.get(MAP_TEXT["Sales Period Position"], ""),   final_boxes["Sales Period Position"],   font_path=FONT_REGULAR_PATH, max_font=28, align="right")

        out_path = OUTPUT_DIR / f"card_cn_{idx+1:03d}.png"
        canvas.convert("RGB").save(out_path, quality=95)
        print("Saved:", out_path)

# =================== 可直接使用的入口 ===================
if __name__ == "__main__":
    # 1) 只需重生訓練資料時開啟：
    run_preprocess(Path("Objects_positions.json"), Path(INITIAL_BOXES_PATH), Path(OUT_CSV), meta_csv=None)

    # 2) 需要重訓時開啟（建議 baseline 調好後重訓）：
    train_model()

    # 3) 出圖
    main_render()


[OK] Saved offsets_for_tf.csv  rows=29
Columns: ['image_id', 'title_len', 'bullets_len', 'bullets_lines', 'vip_digits', 'unit_digits', 'nonvip_digits', 'has_badge', 'Sales Period Position_dx', 'Sales Period Position_dy', 'Sales Period Position_dlogw', 'Sales Period Position_dlogh', 'Brand Position_dx', 'Brand Position_dy', 'Brand Position_dlogw', 'Brand Position_dlogh', 'Product Position_dx', 'Product Position_dy', 'Product Position_dlogw', 'Product Position_dlogh', 'FAB Position_dx', 'FAB Position_dy', 'FAB Position_dlogw', 'FAB Position_dlogh', 'Product Image Position_dx', 'Product Image Position_dy', 'Product Image Position_dlogw', 'Product Image Position_dlogh', 'VIP Price Position_dx', 'VIP Price Position_dy', 'VIP Price Position_dlogw', 'VIP Price Position_dlogh', 'Non VIP Price Position_dx', 'Non VIP Price Position_dy', 'Non VIP Price Position_dlogw', 'Non VIP Price Position_dlogh', 'Original Price Position_dx', 'Original Price Position_dy', 'Original Price Position_dlogw', 'Ori

/opt/anaconda3/envs/boardclick/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - loss: 0.4400 - mean_absolute_error: 0.4151 - mean_squared_error: 0.4400 - val_loss: 0.4119 - val_mean_absolute_error: 0.3985 - val_mean_squared_error: 0.4119
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.4387 - mean_absolute_error: 0.4140 - mean_squared_error: 0.4387 - val_loss: 0.4107 - val_mean_absolute_error: 0.3976 - val_mean_squared_error: 0.4107
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.4374 - mean_absolute_error: 0.4129 - mean_squared_error: 0.4374 - val_loss: 0.4096 - val_mean_absolute_error: 0.3967 - val_mean_squared_error: 0.4096
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.4362 - mean_absolute_error: 0.4118 - mean_squared_error: 0.4362 - val_loss: 0.4085 - val_mean_absolute_error: 0.3959 - val_mean_squared_error: 0.4085
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.4349 - mean_absolute_error: 0.4108 - mean_squared_error: 0.4349 - val_loss: 0.4074 - val_mean_absolute_error: 0.395

In [135]:
# -*- coding: utf-8 -*-
"""
Label Studio → offsets_for_tf.csv → Augmentation → Train(tanh+scaling) → Render
包含：
- [FIX] nonvip_digits 特徵一致
- [FIX] 用 DataFrame 餵 scaler.transform() 避免 sklearn warning
- [NEW] 類別順序 *.classes.json 存取/檢查，避免訓練/推論順序不一致
- [NEW] 模型輸出層 tanh + 標籤縮放（dx,dy=0.3；dlog=0.4）
- [NEW] Augmentation（擾動 baseline offsets）以擴增資料量
"""

import json, csv, re, argparse, sys, math, random
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
import pickle

from PIL import Image, ImageDraw, ImageFont
import textwrap
import unicodedata
import requests
from io import BytesIO

# =================== 全域設定 ===================
USE_MODEL = True
BLEND = 0.2
DRAW_DEBUG_BOXES = True
PRINT_FIRST_PRED = True

# Label scaling（對應 tanh 輸出），推論要乘回
DX_DY_SCALE = 0.1
DLOG_SCALE  = 0.25

EXCEL_PATH = "Board Click SKU.xlsx"
SHEET_USER = "User_Input"
SHEET_IMG  = "SKU_Image"
SHEET_ICON = "SKU_Icon"

TEMPLATE_PATH = "sasa_pink_1280.png"
INITIAL_BOXES_PATH = "Initial_boxes.json"

MODEL_PATH = "layout_model.keras"
SCALER_PATH = "scaler.pkl"

OUT_CSV = "offsets_for_tf.csv"
CLASS_ORDER_FILE = Path(OUT_CSV).with_suffix(".classes.json")  # [NEW]

OUTPUT_DIR = Path("out_cards_cn")
OUTPUT_DIR.mkdir(exist_ok=True)

MAP_TEXT = {
    "Brand Position":          "Brand",
    "VIP Price Position":      "VIP價",
    "Non VIP Price Position":  "優惠價",
    "Original Price Position": "建議價",
    "Product Position":        "Product name",
    "Sales Period Position":   "優惠期",
    "FAB Position":            "FAB",
}
COL_PRODUCT_SKU = "Product SKU"
COL_ICON_NAME   = "Icon"

SKU_IMG_SKU_COL   = "Product SKU"
SKU_IMG_LINK_COL  = "Image"
SKU_ICON_NAME_COL = "Icon"
SKU_ICON_LINK_COL = "Icon Image"

# 字型（請改成你電腦的實際字型路徑）
FONT_REGULAR_PATH = Path.home() / "Library/Fonts/NotoSansCJKsc-Regular.otf"
FONT_BOLD_PATH    = Path.home() / "Library/Fonts/NotoSansCJKsc-Bold.otf"

# =================== 工具 ===================
def safe_str(x): 
    return "" if (x is None or (isinstance(x,float) and pd.isna(x))) else str(x)

def count_digits(s): 
    return sum(ch.isdigit() for ch in safe_str(s))

def read_json_file(path: Path):
    text = path.read_text(encoding="utf-8").strip()
    return json.loads(text)

def parse_ls_rect(item):
    v = item["value"]
    label = v["rectanglelabels"][0] if isinstance(v.get("rectanglelabels"), list) else v.get("labels", [""])[0]
    rect_pct = dict(x=v["x"], y=v["y"], w=v["width"], h=v["height"])
    ow = item.get("original_width") or item.get("image_original_width")
    oh = item.get("original_height") or item.get("image_original_height")
    return label, rect_pct, int(ow), int(oh)

def pct_to_px(rpct, ow, oh):
    return {"x": rpct["x"]/100.0*ow, "y": rpct["y"]/100.0*oh, "w": rpct["w"]/100.0*ow, "h": rpct["h"]/100.0*oh}

def resize_rect(rect, from_w, from_h, to_w, to_h):
    sx, sy = to_w/float(from_w), to_h/float(from_h)
    return {"x": rect["x"]*sx, "y": rect["y"]*sy, "w": rect["w"]*sx, "h": rect["h"]*sy}

def delta_from(gt, init, W, H):
    w0 = max(1e-6, float(init["w"]))
    h0 = max(1e-6, float(init["h"]))
    return {
        "dx":   (float(gt["x"]) - float(init["x"])) / float(W),
        "dy":   (float(gt["y"]) - float(init["y"])) / float(H),
        "dlogw": float(np.log(max(1e-6,float(gt["w"])) / w0)),
        "dlogh": float(np.log(max(1e-6,float(gt["h"])) / h0)),
    }

def _load_font(path, size): return ImageFont.truetype(str(path), size)
def _safe(s): return "" if (s is None or (isinstance(s,float) and pd.isna(s))) else str(s)
def _count_digits(s): return sum(ch.isdigit() for ch in _safe(s))

def _norm_key(x):
    s = "" if pd.isna(x) else str(x)
    s = unicodedata.normalize("NFKC", s)
    return s.strip().casefold()

def _drive_share_to_direct(u: str) -> str:
    if not u: return u
    m = re.search(r"/d/([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    m = re.search(r"[?&]id=([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    return u

def load_image_any(path_or_url: str):
    try:
        s = str(path_or_url).strip()
        if s.startswith("http"):
            resp = requests.get(_drive_share_to_direct(s), timeout=15)
            resp.raise_for_status()
            return Image.open(BytesIO(resp.content)).convert("RGBA")
        p = Path(s)
        if p.exists(): return Image.open(p).convert("RGBA")
    except Exception as e:
        print("[WARN] cannot load image:", path_or_url, e)
    return None

def paste_image_into_box(canvas_rgba, path_or_url, box, padding=6):
    im = load_image_any(path_or_url)
    if im is None: return
    x,y,w,h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    w2,h2 = max(1,w-padding*2), max(1,h-padding*2)
    ratio = min(w2/im.width, h2/im.height)
    im = im.resize((max(1,int(im.width*ratio)), max(1,int(im.height*ratio))), Image.LANCZOS)
    ox = x+(w-im.width)//2
    oy = y+(h-im.height)//2
    canvas_rgba.alpha_composite(im,(ox,oy))

def draw_text_in_box(draw,text,box,font_path,max_font=64,min_font=16,align="left",line_spacing=1.15):
    if not text or str(text).strip()=="": return
    x,y,w,h = int(box["x"]),int(box["y"]),int(box["w"]),int(box["h"])
    text = str(text)
    for fs in range(max_font,min_font-1,-2):
        font = _load_font(font_path, fs)
        approx_chars = max(1,int(w/(fs*0.55)))
        lines=[]
        for raw in text.split("\n"):
            lines += (textwrap.wrap(raw,width=approx_chars) if approx_chars>1 else [raw])
        bboxes=[draw.textbbox((0,0),ln,font=font) for ln in lines]
        total_h=int(sum(bb[3]-bb[1] for bb in bboxes)+(len(lines)-1)*fs*(line_spacing-1))
        if total_h<=h:
            cur_y=y+(h-total_h)//2
            for ln in lines:
                bb=draw.textbbox((0,0),ln,font=font)
                lw=bb[2]-bb[0]
                cur_x=x if align=="left" else (x+(w-lw)//2 if align=="center" else x+(w-lw))
                draw.text((cur_x,cur_y),ln,font=font,fill=(0,0,0,255))
                cur_y+=int(fs*line_spacing)
            return

def _stroke(draw, box, color=(0,0,0,255), width=2):
    x,y,w,h = int(box["x"]),int(box["y"]),int(box["w"]),int(box["h"])
    draw.rectangle([x,y,x+w,y+h], outline=color, width=width)

def _clip(v, lo, hi): return max(lo, min(hi,v))

# =================== 特徵 ===================
FEATURE_ORDER = ["title_len","bullets_len","bullets_lines","vip_digits","unit_digits","nonvip_digits","has_badge"]

def build_features(meta_row: dict):
    return {
        "title_len": len(safe_str(meta_row.get("title",""))),
        "bullets_len": len(safe_str(meta_row.get("bullets",""))),
        "bullets_lines": len([b for b in re.split(r"[;\n]", safe_str(meta_row.get("bullets",""))) if b.strip()]),
        "vip_digits": count_digits(meta_row.get("vip_price","")),
        "unit_digits": count_digits(meta_row.get("unit_price","")),
        "nonvip_digits": count_digits(meta_row.get("non_vip_price","")),
        "has_badge": int(str(meta_row.get("has_badge",0)).strip() not in ["","0","False","false"]),
    }

def build_features_from_row(row: pd.Series, resolved_icon_path: str) -> dict:
    return {
        "title_len": len(_safe(row.get(MAP_TEXT["Product Position"],""))),
        "bullets_len": len(_safe(row.get(MAP_TEXT["FAB Position"],""))),
        "bullets_lines": len([b for b in _safe(row.get(MAP_TEXT["FAB Position"],"")).split(";") if b.strip()]),
        "vip_digits": _count_digits(row.get(MAP_TEXT["VIP Price Position"],"")),
        "unit_digits": _count_digits(row.get(MAP_TEXT["Non VIP Price Position"],"")),
        "nonvip_digits": _count_digits(row.get(MAP_TEXT["Non VIP Price Position"],"")),  # [FIX]
        "has_badge": int(Path(_safe(resolved_icon_path)).exists()) if resolved_icon_path else 0,
    }

# =================== Preprocess → CSV ===================
def run_preprocess(label_studio_json: Path, initial_boxes_json: Path, out_csv: Path, meta_csv: Path|None=None):
    init_cfg = read_json_file(initial_boxes_json)
    Tw, Th = int(init_cfg["canvas"]["width"]), int(init_cfg["canvas"]["height"])
    init_boxes = init_cfg["boxes"]
    classes = list(init_boxes.keys())

    tasks = read_json_file(label_studio_json)
    rows=[]
    for t in tasks:
        annos = t.get("annotations") or []
        if not annos: continue
        res = annos[0].get("result", [])
        gt_scaled={}
        for r in res:
            if r.get("type") not in ("rectanglelabels","rectangles"): continue
            label, rpct, ow, oh = parse_ls_rect(r)
            if label not in classes: continue
            rect_px = pct_to_px(rpct, ow, oh)
            rect_tpl = resize_rect(rect_px, ow, oh, Tw, Th)
            gt_scaled[label]=rect_tpl
        row={"image_id":t.get("id")}
        feats=build_features({})
        row.update(feats)
        for cls in classes:
            init=init_boxes[cls]; gt=gt_scaled.get(cls)
            d=delta_from(gt,init,Tw,Th) if gt else {"dx":0,"dy":0,"dlogw":0,"dlogh":0}
            for k,v in d.items(): row[f"{cls}_{k}"]=v
        rows.append(row)
    df=pd.DataFrame(rows)
    df.to_csv(out_csv,index=False)
    class_file = out_csv.with_suffix(".classes.json")
    json.dump(classes, open(class_file, "w", encoding="utf-8"), ensure_ascii=False, indent=2) # [NEW]
    print(f"[OK] Saved {out_csv} rows={len(df)}")
    return df

# =================== Augmentation ===================
def augment_offsets(df, classes, num_aug=10):
    aug_rows=[]
    for _, row in df.iterrows():
        for _ in range(num_aug):
            new_row=row.copy()
            for cls in classes:
                new_row[f"{cls}_dx"]=row[f"{cls}_dx"]+random.uniform(-0.15,0.15)
                new_row[f"{cls}_dy"]=row[f"{cls}_dy"]+random.uniform(-0.15,0.15)
                new_row[f"{cls}_dlogw"]=row[f"{cls}_dlogw"]+random.uniform(-0.2,0.2)
                new_row[f"{cls}_dlogh"]=row[f"{cls}_dlogh"]+random.uniform(-0.2,0.2)
            aug_rows.append(new_row)
    return pd.concat([df,pd.DataFrame(aug_rows)],ignore_index=True)

# =================== 訓練（tanh + label scaling） ===================
def train_model():
    df=pd.read_csv(OUT_CSV)
    classes=json.load(open(CLASS_ORDER_FILE, "r", encoding="utf-8"))
    df_aug=augment_offsets(df,classes,num_aug=10)
    print(f"[INFO] Augmented {len(df)}→{len(df_aug)}")
    X=df_aug.drop(columns=[c for c in df_aug.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh")) or c=="image_id"])
    Y=df_aug[[c for c in df_aug.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh"))]]
    Y_scaled=Y.copy()
    for c in Y.columns:
        if c.endswith(("_dx","_dy")): Y_scaled[c]=Y[c]/DX_DY_SCALE
        else: Y_scaled[c]=Y[c]/DLOG_SCALE
    Xtr,Xv,Ytr,Yv=train_test_split(X,Y_scaled,test_size=0.2,random_state=42)
    scaler=StandardScaler(); Xtr=scaler.fit_transform(Xtr); Xv=scaler.transform(Xv)
    model=tf.keras.Sequential([
        tf.keras.layers.Dense(128,activation='relu',input_shape=(Xtr.shape[1],)),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(Ytr.shape[1],activation='tanh')  # [NEW]
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),loss='mse',metrics=[MeanSquaredError(),MeanAbsoluteError()])
    model.fit(Xtr,Ytr,validation_data=(Xv,Yv),epochs=50,batch_size=16,verbose=1)
    model.save(MODEL_PATH); pickle.dump(scaler,open(SCALER_PATH,"wb"))
    print("[OK] Model saved")

# =================== 幾何 / 佈局 ===================
def apply_offset(init_box, delta, W, H):
    dx, dy, dlogw, dlogh = map(float, delta)
    # （可選）安全範圍限制
    dx = _clip(dx, -0.5, 0.5)          # [NEW] 避免飛太遠
    dy = _clip(dy, -0.5, 0.5)          # [NEW]
    sx = math.exp(_clip(dlogw, math.log(0.5), math.log(2.0)))  # [NEW]
    sy = math.exp(_clip(dlogh, math.log(0.5), math.log(2.0)))  # [NEW]
    x = init_box["x"] + dx * W
    y = init_box["y"] + dy * H
    w = max(24, init_box["w"] * sx)
    h = max(24, init_box["h"] * sy)
    x = _clip(x, 0, W - w)             # [NEW] 裁進畫布
    y = _clip(y, 0, H - h)             # [NEW]
    return {"x": x, "y": y, "w": w, "h": h}

# =================== 模型 + Scaler 載入 ===================
def load_template_and_boxes():
    tpl = Image.open(TEMPLATE_PATH).convert("RGBA")
    tw, th = tpl.size
    init_cfg = json.loads(Path(INITIAL_BOXES_PATH).read_text(encoding="utf-8"))
    cw, ch = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    if (tw, th) != (cw, ch):
        sx, sy = tw/float(cw), th/float(ch)
        for _, r in init_cfg["boxes"].items():
            r["x"], r["y"] = r["x"]*sx, r["y"]*sy
            r["w"], r["h"] = r["w"]*sx, r["h"]*sy
        init_cfg["canvas"]["width"], init_cfg["canvas"]["height"] = tw, th
        print(f"[INFO] Scaled initial boxes to {tw}x{th}")
    return tpl, init_cfg

def load_model_and_scaler():
    model = tf.keras.models.load_model(MODEL_PATH) if Path(MODEL_PATH).exists() else None
    scaler = pickle.load(open(SCALER_PATH,"rb")) if Path(SCALER_PATH).exists() else None
    if model is None:  print("[WARN] MODEL not found → zero offsets")
    if scaler is None: print("[WARN] SCALER not found → use raw features")
    return model, scaler

# =================== 渲染主程式 ===================
def main_render():
    # 讀 Excel
    xls = pd.ExcelFile(EXCEL_PATH)
    df_user = pd.read_excel(xls, sheet_name=SHEET_USER)
    df_img  = pd.read_excel(xls, sheet_name=SHEET_IMG)
    df_icon = pd.read_excel(xls, sheet_name=SHEET_ICON)

    # 建查表
    img_lookup  = {_norm_key(r[SKU_IMG_SKU_COL]): str(r[SKU_IMG_LINK_COL]).strip() for _, r in df_img.iterrows()}
    icon_lookup = {_norm_key(r[SKU_ICON_NAME_COL]): str(r[SKU_ICON_LINK_COL]).strip() for _, r in df_icon.iterrows()}

    # 載入模板與模型
    template_rgba, init_cfg = load_template_and_boxes()
    W, H = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    INIT_BOXES = init_cfg["boxes"]

    # [FIX] 讀回訓練時類別順序，並檢查集合一致
    if CLASS_ORDER_FILE.exists():
        train_classes = json.load(open(CLASS_ORDER_FILE, "r", encoding="utf-8"))
        if set(train_classes) != set(INIT_BOXES.keys()):
            raise ValueError(f"Classes mismatch.\nTrain: {train_classes}\nInfer: {list(INIT_BOXES.keys())}")
        CLASSES = train_classes[:]
    else:
        CLASSES = list(INIT_BOXES.keys())

    model, scaler = load_model_and_scaler()

    # 逐列生成
    for idx, row in df_user.iterrows():
        sku_key  = _norm_key(row.get(COL_PRODUCT_SKU, ""))
        icon_key = _norm_key(row.get(COL_ICON_NAME, ""))
        prod_img_path = img_lookup.get(sku_key, "")
        icon_img_path = icon_lookup.get(icon_key, "")

        # 特徵 → DataFrame 給 scaler（避免 sklearn warning）
        feats = build_features_from_row(row, resolved_icon_path=icon_img_path)
        X_df = pd.DataFrame([feats], columns=FEATURE_ORDER).astype(float)  # [FIX]
        X_in = scaler.transform(X_df) if scaler is not None else X_df.values

        # baseline offsets (全 0)
        deltas_base = {cls: [0.0,0.0,0.0,0.0] for cls in CLASSES}

        # 預測 offsets（tanh 輸出需乘回 label scaling）
        if USE_MODEL and model is not None:
            pred = model.predict(X_in, verbose=0)[0]
            deltas_pred = {}
            for i, cls in enumerate(CLASSES):
                dx = float(pred[i*4+0]) * DX_DY_SCALE
                dy = float(pred[i*4+1]) * DX_DY_SCALE
                dw = float(pred[i*4+2]) * DLOG_SCALE
                dh = float(pred[i*4+3]) * DLOG_SCALE
                deltas_pred[cls] = [dx,dy,dw,dh]
        else:
            deltas_pred = deltas_base

        if idx == 0 and PRINT_FIRST_PRED and USE_MODEL and (model is not None):  # [FIX]
            print("Pred deltas (first row, scaled back):", pred[:min(12, len(pred))])

        # blend baseline 與 model
        deltas_final = {}
        for cls in CLASSES:
            db = deltas_base[cls]; dp = deltas_pred[cls]
            deltas_final[cls] = [(1-BLEND)*db[i] + BLEND*dp[i] for i in range(4)]

        final_boxes = {cls: apply_offset(INIT_BOXES[cls], deltas_final[cls], W, H) for cls in CLASSES}

        # 繪圖
        canvas = template_rgba.copy()
        draw = ImageDraw.Draw(canvas)

        paste_image_into_box(canvas, prod_img_path, final_boxes["Product Image Position"])
        paste_image_into_box(canvas, icon_img_path,  final_boxes["Icon Position"])

        draw_text_in_box(draw, row.get(MAP_TEXT["Brand Position"], ""),          final_boxes["Brand Position"],          font_path=FONT_BOLD_PATH,   max_font=72)
        draw_text_in_box(draw, row.get(MAP_TEXT["Product Position"], ""),        final_boxes["Product Position"],        font_path=FONT_REGULAR_PATH,max_font=64)
        fab_text = str(row.get(MAP_TEXT["FAB Position"], "") or "").replace(";", "\n")
        draw_text_in_box(draw, fab_text,                                        final_boxes["FAB Position"],            font_path=FONT_REGULAR_PATH,max_font=44)
        draw_text_in_box(draw, row.get(MAP_TEXT["VIP Price Position"], ""),      final_boxes["VIP Price Position"],      font_path=FONT_BOLD_PATH,   max_font=100)
        draw_text_in_box(draw, row.get(MAP_TEXT["Non VIP Price Position"], ""),  final_boxes["Non VIP Price Position"],  font_path=FONT_REGULAR_PATH,max_font=36)
        draw_text_in_box(draw, row.get(MAP_TEXT["Original Price Position"], ""), final_boxes["Original Price Position"], font_path=FONT_REGULAR_PATH,max_font=32)
        draw_text_in_box(draw, row.get(MAP_TEXT["Sales Period Position"], ""),   final_boxes["Sales Period Position"],   font_path=FONT_REGULAR_PATH,max_font=28, align="right")

        if DRAW_DEBUG_BOXES:
            for cls, box in final_boxes.items():
                _stroke(draw, box, (255,0,0,180), 2)

        out_path = OUTPUT_DIR / f"card_cn_{idx+1:03d}.png"
        canvas.convert("RGB").save(out_path, quality=95)
        print("Saved:", out_path)

# =================== 主程式 ===================
if __name__ == "__main__":
    # --- 首次完整流程（生資料 → 訓練 → 出圖），跑完再註解 ---
    run_preprocess(Path("Objects_positions.json"), Path(INITIAL_BOXES_PATH), Path(OUT_CSV), meta_csv=None)
    train_model()
    # main_render()

    # --- 日常出圖（已經有模型與 scaler） ---
    main_render()


[OK] Saved offsets_for_tf.csv rows=29
[INFO] Augmented 29→319
Epoch 1/50


/opt/anaconda3/envs/boardclick/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.6948 - mean_absolute_error: 1.0028 - mean_squared_error: 1.6948 - val_loss: 1.8303 - val_mean_absolute_error: 1.0315 - val_mean_squared_error: 1.8303
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6780 - mean_absolute_error: 0.9977 - mean_squared_error: 1.6780 - val_loss: 1.8129 - val_mean_absolute_error: 1.0262 - val_mean_squared_error: 1.8129
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6617 - mean_absolute_error: 0.9927 - mean_squared_error: 1.6617 - val_loss: 1.7958 - val_mean_absolute_error: 1.0210 - val_mean_squared_error: 1.7958
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6458 - mean_absolute_error: 0.9878 - mean_squared_error: 1.6458 - val_loss: 1.7792 - val_mean_absolute_error: 1.0159 - val_mean_squared_error: 1.7792
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6305 - mean_absolute_error: 0.9831 - mean_squared_error: 1.6305 - val_loss: 1.7629 - val_mean_absolute_error: 1